In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from collections import Counter # Thêm thư viện này để đếm (bỏ phiếu)

# 1️⃣ Load dữ liệu từ file Iris.csv
df = pd.read_csv("Iris.csv")

# Kiểm tra tên cột
print("Tên các cột:", df.columns.tolist())
print(df.head())

# Giả sử cột label là 'species'
X = df.iloc[:, :-1].values  # 4 cột đầu là features
y = df.iloc[:, -1].values   # cột cuối là label

# 2️⃣ Kiểm tra dữ liệu thiếu
print("\nCó giá trị thiếu không:")
print(df.isnull().sum())

# 3️⃣ Chia dữ liệu train/test (70% train, 30% test, giữ cân bằng lớp)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print("Số mẫu train:", X_train.shape[0])
print("Số mẫu test:", X_test.shape[0])

# 4️⃣ Lớp KNN
class KNN:
    def __init__(self, k=5):
        self.k = k

    def fit(self, X, y):
        # "Huấn luyện" KNN chỉ là lưu trữ dữ liệu
        self.X_train = X
        self.y_train = y

    def _euclidean_distance(self, x1, x2):
        # Hàm tính khoảng cách
        return np.sqrt(np.sum((x1 - x2)**2))

    def _predict_single(self, x):
        # 1. Tính khoảng cách từ x đến tất cả các điểm train
        distances = [self._euclidean_distance(x, x_train) for x_train in self.X_train]
        
        # 2. Lấy chỉ số của k điểm gần nhất
        k_indices = np.argsort(distances)[:self.k]
        
        # 3. Lấy nhãn của k điểm đó
        k_nearest_labels = [self.y_train[i] for i in k_indices]
        
        # 4. Bỏ phiếu: Lấy nhãn xuất hiện nhiều nhất
        most_common = Counter(k_nearest_labels).most_common(1)
        return most_common[0][0]

    def predict(self, X):
        return np.array([self._predict_single(x) for x in X])

# 5️⃣ Huấn luyện và dự đoán
knn = KNN(k=5) # Chọn k=5
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

# 6️⃣ In kết quả
print("Độ chính xác trên tập test:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Tên các cột: ['Id', 'SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm', 'Species']
   Id  SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm      Species
0   1            5.1           3.5            1.4           0.2  Iris-setosa
1   2            4.9           3.0            1.4           0.2  Iris-setosa
2   3            4.7           3.2            1.3           0.2  Iris-setosa
3   4            4.6           3.1            1.5           0.2  Iris-setosa
4   5            5.0           3.6            1.4           0.2  Iris-setosa

Có giá trị thiếu không:
Id               0
SepalLengthCm    0
SepalWidthCm     0
PetalLengthCm    0
PetalWidthCm     0
Species          0
dtype: int64
Số mẫu train: 105
Số mẫu test: 45
Độ chính xác trên tập test: 1.0

Classification Report:
                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        15
Iris-versicolor       1.00      1.00      1.00        15
 Iris-virginica       1.00 

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 1️⃣ Load dữ liệu Letter
df_letter = pd.read_csv("letter-recognition.data", header=None)
X = df_letter.iloc[:, 1:].values
y = df_letter.iloc[:, 0].values

# 2️⃣ Chia dữ liệu train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# 3️⃣ Lớp kNN vector hóa với NumPy
class KNN:
    def __init__(self, k=5):
        self.k = k

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        # Tính khoảng cách Euclidean giữa tất cả X và X_train (vector hóa)
        # Dùng công thức ||a-b||^2 = ||a||^2 + ||b||^2 - 2*a.b
        X_norm = np.sum(X**2, axis=1).reshape(-1, 1)
        X_train_norm = np.sum(self.X_train**2, axis=1).reshape(1, -1)
        distances = np.sqrt(X_norm + X_train_norm - 2 * X.dot(self.X_train.T))

        # Lấy k nearest neighbors và dự đoán
        k_indices = np.argsort(distances, axis=1)[:, :self.k]
        k_nearest_labels = self.y_train[k_indices]

        # Dự đoán nhãn xuất hiện nhiều nhất
        y_pred = []
        for labels in k_nearest_labels:
            values, counts = np.unique(labels, return_counts=True)
            y_pred.append(values[np.argmax(counts)])
        return np.array(y_pred)

# 4️⃣ Huấn luyện và dự đoán
knn_letter = KNN(k=5)
knn_letter.fit(X_train, y_train)
y_pred = knn_letter.predict(X_test)

# 5️⃣ In kết quả
print("Accuracy trên Letter dataset:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Accuracy trên Letter dataset: 0.947

Classification Report:
              precision    recall  f1-score   support

           A       0.99      0.99      0.99       237
           B       0.90      0.94      0.92       230
           C       0.93      0.96      0.95       221
           D       0.88      0.98      0.93       242
           E       0.86      0.93      0.90       230
           F       0.95      0.93      0.94       232
           G       0.95      0.90      0.92       232
           H       0.84      0.89      0.87       220
           I       0.96      0.96      0.96       226
           J       0.96      0.94      0.95       224
           K       0.91      0.87      0.89       222
           L       0.98      0.98      0.98       228
           M       0.98      0.97      0.97       238
           N       0.96      0.93      0.94       235
           O       0.90      0.93      0.91       226
           P       0.97      0.95      0.96       241
           Q       0.